## This model has been trained on Google Colab and as part of Udacity's *Introduction to Deep Learning using Pytorch* course. This notebook is built on top of a guidline template.


### Premodel steps to install Google Drive, Pytorch and the flowers dataset

In [0]:
# Load the Drive helper and mount
from google.colab import drive

drive.mount('/content/drive')

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58636000 @  0x7f03237cc2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
fastai 1.0.41 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.


In [0]:
# Load flowers dataset into temporary storage
import requests

url = 'https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip'
target_path = 'flowers_train.zip'

response = requests.get(url, stream=True)
handle = open(target_path, "wb")
for chunk in response.iter_content(chunk_size=512):
   if chunk:  # filter out keep-alive new chunks
       handle.write(chunk)
handle.close()

In [0]:
# Unzip downloaded dataset
!unzip flowers_train.zip

In [4]:
!pip install -U pillow

Requirement already up-to-date: pillow in /usr/local/lib/python3.6/dist-packages (5.4.1)


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.nn import functional as F
from torchvision import datasets, models
from torchvision.transforms import transforms

import numpy as np
import helper
import PIL

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
test_dir = data_dir + '/valid'

In [0]:
# Define transforms for the training and validation sets
train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.CenterCrop(224), # Crop image to desired size
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])


# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# Define dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [0]:
import json

with open('drive/My Drive/Colab Notebooks/pytorch-challenge/cat_to_name.json',
          'r') as f:
    category_to_name = json.load(f)
    print(category_to_name['2'])

hard-leaved pocket orchid


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [0]:
# Use Densenet121
model = models.densenet121(pretrained=True)

# Freeze Densenet121 feature parameters
for param in model.parameters():
    param.requires_grad = False

# Define new classifier that fits our model
model.classifier = nn.Sequential(
    nn.Linear(1024, 500),
    nn.ReLU(),
    nn.Linear(500, 500),
    nn.ReLU(),
    nn.Linear(500, 102),
    nn.LogSoftmax(dim=1)
)

# Define loss criterion
criterion = nn.NLLLoss()
# Only give the optimizer the classifier's parameters
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1, last_epoch=-1)

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:01<00:00, 21279604.61it/s]


In [0]:
# Load presaved model (If exists)
state_dict = torch.load('drive/My Drive/Colab Notebooks/pytorch-challenge/models/model_final.pt')
model.load_state_dict(state_dict)

In [0]:
# Get device on which to train the model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [0]:
# Train the model
epochs = 5
min_loss = 0.0

for e in range(epochs):
    train_loss, iteration_count, accuracy = 0, 0, 0.0
    
    #model.to(device)
    model.train()
    scheduler.step()
    
    for images, labels in train_loader:
        iteration_count += 1
        # Move tensors to GPU
        images, labels = images.to(device), labels.to(device)
            
        optimizer.zero_grad()
        output = model.forward(images)
        loss = criterion(output, labels)
        
        loss.backward()    # Clalculate gradient
        optimizer.step()   # Update weights
        train_loss += loss.item() * images.size(0)
        
        if iteration_count % 10 == 0: 
          print(iteration_count, end=' ')
     
    # Evaluating the model
    valid_loss = 0
    with torch.no_grad():
        model.eval()
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            log_output = model.forward(images)
            loss = criterion(log_output, labels)
            valid_loss += loss.item() * images.size(0)

            # Getting class with max prob for each img in output
            output = torch.exp(log_output)
            equality = (labels.data == output.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()
        
    # Initialize minumum loss if it's the first epoch
    if e == 0:
      min_loss = valid_loss
    # Save the model if it has the lowest loss
    elif valid_loss < min_loss:
      min_loss = valid_loss
      torch.save(model.state_dict(), 
                 'drive/My Drive/Colab Notebooks/pytorch-challenge/models/model_final.pt')

        
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    accuracy = accuracy/len(test_loader)      
    
    print('\nEpoch', e,
          'Train Loss:', train_loss,
          'Validation Loss', valid_loss,
          '\nAccuracy:', accuracy * 100, '\n')
    

10 20 30 40 50 60 70 80 90 100 
Epoch 0 Train Loss: 2.83782660306155 Validation Loss 1.4340254848044192 
Accuracy: tensor(63.6352) 

10 20 30 40 50 60 70 80 90 100 
Epoch 1 Train Loss: 0.9571096130779811 Validation Loss 0.9005076155102923 
Accuracy: tensor(73.6645) 

10 20 30 40 50 60 70 80 90 100 
Epoch 2 Train Loss: 0.5937219658759633 Validation Loss 0.6270119645484794 
Accuracy: tensor(83.9477) 

10 20 30 40 50 60 70 80 90 100 
Epoch 3 Train Loss: 0.4533106605357448 Validation Loss 0.5696239182884943 
Accuracy: tensor(82.5321) 



## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
# Save the checkpoint 
torch.save(model.state_dict(), 'path_to_saved_model.pt')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
state_dict = torch.load('path_to_saved_model.pt')
model.load_state_dict(state_dict)